In [64]:
# spark.stop()

In [50]:
import os
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, BooleanType

In [51]:
spark = (
    SparkSession
    .builder
    .master("spark://spark-master:7077")
    .appName("Beginner")
    .config("spark.jars", "https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-latest.jar") # GCS Connector
    .getOrCreate()
)

# Google Cloud Service Account Credentials
spark._jsc.hadoopConfiguration().set("google.cloud.auth.service.account.json.keyfile",os.environ.get("GOOGLE_APPLICATION_CREDENTIALS"))

spark

In [54]:
!rm -rf ./data

In [55]:
# Define the schema for the DataFrame
schema1 = StructType([
    StructField("Name", StringType(), True),  # Column 1: Name (String)
    StructField("Age", IntegerType(), True),  # Column 2: Age (Integer)
    StructField("City", StringType(), True)   # Column 3: City (String)
])

# Sample data with a correction for the Age column (casting '25' to an integer)
data1 = [("Alice", 30, "New York"), 
        ("Bob", 25, "San Francisco"),  # Corrected age data type to integer
        ("Catherine", 28, "Chicago")]

# Create DataFrame using the defined schema1
df = spark.createDataFrame(data1, schema1)

# Show the DataFrame to verify the data
df.show()

# Save the DataFrame as a Parquet file
df.write.parquet("./data/output-1")

+---------+---+-------------+
|     Name|Age|         City|
+---------+---+-------------+
|    Alice| 30|     New York|
|      Bob| 25|San Francisco|
|Catherine| 28|      Chicago|
+---------+---+-------------+



In [56]:
schema2 = StructType([
    StructField("Name", StringType(), True),  # Column 1: Name (String)
    StructField("Age", StringType(), True),  # Column 2: Age (String)
    StructField("City", StringType(), True)   # Column 3: City (String)
])

# Sample data with Age as a string
data2 = [("Alice", "30", "New York")]

# Create DataFrame using the defined schema (Age as a string)
df = spark.createDataFrame(data2, schema2)

df.write.parquet("./data/output-2")

In [63]:
new_df = (
    spark
    .read
    .option("mergeSchema", "true")
    .parquet("./data/*")
)
new_df.printSchema()

Py4JJavaError: An error occurred while calling o485.parquet.
: org.apache.spark.SparkException: Failed merging schema:
root
 |-- Name: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- City: string (nullable = true)

	at org.apache.spark.sql.execution.datasources.SchemaMergeUtils$.$anonfun$mergeSchemasInParallel$5(SchemaMergeUtils.scala:100)
	at org.apache.spark.sql.execution.datasources.SchemaMergeUtils$.$anonfun$mergeSchemasInParallel$5$adapted(SchemaMergeUtils.scala:95)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:198)
	at org.apache.spark.sql.execution.datasources.SchemaMergeUtils$.mergeSchemasInParallel(SchemaMergeUtils.scala:95)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.mergeSchemasInParallel(ParquetFileFormat.scala:493)
	at org.apache.spark.sql.execution.

In [60]:
new_df.show()

25/04/27 21:13:51 WARN TaskSetManager: Lost task 2.0 in stage 12.0 (TID 33, 172.18.0.5, executor 3): org.apache.spark.sql.execution.QueryExecutionException: Parquet column cannot be converted in file file:///opt/workspace/data/output-2/part-00007-3b86f526-3c78-4a69-a77d-6a98f7bd0f21-c000.snappy.parquet. Column: [Age], Expected: int, Found: BINARY
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:179)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:93)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:490)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.columnartorow_nextBatch_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43

Py4JJavaError: An error occurred while calling o453.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 12.0 failed 4 times, most recent failure: Lost task 2.3 in stage 12.0 (TID 37, 172.18.0.7, executor 0): org.apache.spark.sql.execution.QueryExecutionException: Parquet column cannot be converted in file file:///opt/workspace/data/output-2/part-00007-3b86f526-3c78-4a69-a77d-6a98f7bd0f21-c000.snappy.parquet. Column: [Age], Expected: int, Found: BINARY
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:179)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:93)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:490)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.columnartorow_nextBatch_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$